In [ ]:
import xml.etree.ElementTree as ET
import cv2
from PIL import Image
from os import listdir as ld
from os.path import join as pj
import numpy as np
from scipy import ndimage
import h5py
from scipy import sparse
from tqdm import tqdm

from IO.loader import parse_annotations, file_id
from evaluation.classification.statistics import compute_anno_stats, compute_average_size, compute_size_correction
from evaluation.classification.visualize import plot_size_of_anno, plot_size_by_class_of_anno
from utils.crop import *
from utils.annotate import *
from IO.build_ds import build_classification_ds

import matplotlib.pyplot as plt
plt.style.use("dark_background")
%matplotlib inline

### Make data and Visualize

In [ ]:
root = "/home/tanida/workspace/Insect_Phenology_Detector/data"
anno_folders = ["annotations_20200806"]
annos = []
for anno_folder in anno_folders:
    annos_name = ld(pj(root, anno_folder))
    annos.extend([pj(root, anno_folder, x) for x in annos_name])
imgs  = ld(pj(root, "refined_images"))
imgs  = [pj(root, "refined_images", x) for x in imgs if x != ".ipynb_checkpoints"]

In [ ]:
images = {file_id(im):np.array(Image.open(im)) for im in imgs}
annotations = {idx: list(filter(lambda x:idx in x, annos)) for idx in images}
annotations = {k:v for  k,v in annotations.items() if len(v)>0}

anno = {}
for k,v in annotations.items():
    anno[k]=[]
    for x in filter(lambda x:x.endswith(".xml"), v):
        anno[k].extend(parse_annotations(x))

In [ ]:
lbl_to_name = {
    0: 'Coleoptera', 
    1: 'Diptera', 
    2: 'Ephemeridae', 
    3: 'Ephemeroptera', 
    4: 'Hemiptera', 
    5: 'Hymenoptera', 
    6: 'Lepidoptera', 
    7: 'Megaloptera', 
    8: 'Plecoptera', 
    9: 'Trichoptera', 
    10: 'Unknown', 
    11: ']', 
    12: 'unknown'
}

In [ ]:
idxs, counts = np.unique(Y, return_counts=True)
idxs = np.asarray([lbl_to_name[idx] for idx in idxs])
for idx, count in zip(idxs, counts):
    print("{}: {}".format(idx, count))

In [ ]:
idx = 0

In [ ]:
img = X[idx]
label = lbl_to_name[Y[idx]]
plt.imshow(img)
print(label)
idx += 1

### Classify failed Visualize

In [ ]:
import cv2
from os import getcwd as cwd
from os.path import join as pj
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from model.resnet.resnet import ResNet
%matplotlib inline

In [ ]:
lbl_to_name = {
    0: 'Diptera', 
    1: 'Ephemeridae', 
    2: 'Ephemeroptera', 
    3: 'Lepidoptera', 
    4: 'Plecoptera', 
    5: 'Trichoptera', 
}

In [ ]:
exp_name = "master_paper/resnet50/b20_lr1e-5/crossvalid_20200806_OS_All5to6"
fail_count = pd.read_csv(pj(cwd(), "figure/classification", exp_name, "fail_count.csv"))
all_data_path = pj(cwd(), "data/all_classification_data/classify_insect_std_20200806")
model_path = pj(cwd(), "output_model/classification", exp_name, "best.pth")
with h5py.File(all_data_path) as f:
    X = f["X"][:]
    Y = f["Y"][:]
#model = ResNet("resnet50", 6, pretrain=True, param_freeze=False, use_dropout=True, activation_function="ReLU", decoder=None).cuda()
#model.load_state_dict(torch.load(model_path))

In [ ]:
fail_count = np.array(list(fail_count["fail_count"]))
fail_count.shape

In [ ]:
fail_count_filter = fail_count > 1

In [ ]:
X_filtered_fail_count = X[fail_count_filter]
Y_filtered_fail_count = Y[fail_count_filter]
Y_filtered_fail_count.shape

In [ ]:
idx, count = np.unique(Y_filtered_fail_count, return_counts=True)
for insect_id, insect_count in zip(idx, count):
    print("name = {}, fails = {}".format(lbl_to_name[insect_id], insect_count))

In [ ]:
diptera_filter = Y_filtered_fail_count == 0
ephemeridae_filter = Y_filtered_fail_count == 1
ephemeroptera_filter = Y_filtered_fail_count == 2
lepidoptera_filter = Y_filtered_fail_count == 3
plecoptera_filter = Y_filtered_fail_count == 4
trichoptera_filter = Y_filtered_fail_count == 5

In [ ]:
insect_filter = plecoptera_filter

In [ ]:
filtered_X = X_filtered_fail_count[insect_filter]
filtered_Y = Y_filtered_fail_count[insect_filter]
filtered_Y.shape

In [ ]:
idx = 0

In [ ]:
# get data
img = filtered_X[idx]
gt_label = lbl_to_name[filtered_Y[idx]]

# estimate label
"""
x = img.astype("float32")
x = cv2.normalize(x, x, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
x = x.transpose(2,0,1).astype("float32")
x = torch.from_numpy(x).clone()
out = model(x[None, :].cuda())
out = torch.nn.functional.softmax(out[None, :], dim=1)
out_label = torch.max(out, 1)[1]
out_label = out_label[0].cpu().numpy()
out_label = lbl_to_name[int(out_label)]
"""

# plot
plt.imshow(img)
print("gt_label  = {}".format(gt_label))
#print("out_label = {}".format(out_label))
idx += 1